In [1]:
import pandas as pd
import numpy as np
import wandb
import accelerate
import bitsandbytes
import datasets
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import tokenizers
import torch
import transformers
import trl
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import TrainingArguments, set_seed, get_constant_schedule
from trl import SFTTrainer, setup_chat_format, DataCollatorForCompletionOnlyLM
from datasets import load_dataset
import uuid
from datasets import DatasetDict, Dataset

[2024-06-30 21:03:23,894] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  please install triton==1.0.0 if you want to use sparse attention


In [2]:
# Load the dataset using datasets library
ds = load_dataset("FinGPT/fingpt-fiqa_qa")

total_df = ds['train'].train_test_split(test_size = 0.2, seed = 42)

total_df = DatasetDict({
    'train': total_df['train'],
    'test' : total_df['test']
})

def format_data(example):
  return [
      {'content': example['instruction'] + "\n" + example['input'], 'role':'user'},
      {'content': example['output'], 'role': 'assistant'}
  ]


format_train = total_df['train'].map(lambda x: {'messages': format_data(x)})
format_test = total_df['test'].map(lambda x: {'messages': format_data(x)})

final_dataset = DatasetDict({
    'train': Dataset.from_dict({'messages':format_train['messages']}),
    'test': Dataset.from_dict({'messages': format_test['messages']})
})

final_dataset

Map:   0%|          | 0/13688 [00:00<?, ? examples/s]

Map:   0%|          | 0/3422 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 13688
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 3422
    })
})

In [3]:
train_df = pd.DataFrame(format_train)

test_df = pd.DataFrame(format_test)


In [5]:
tokenizer = AutoTokenizer.from_pretrained("/home/trkosire/LLM_RESEARCH/Finance/out_OpenELM-270M_LR-5e-05_OA_bf130067-0855-427c-ba89-f63b91981af7/checkpoint-3250")

token_lengths = test_df['output'].apply(lambda x: len(tokenizer.tokenize(x)))

avg_of_output_token_count = token_lengths.mean()

print(f"\n avg_of_output_token_count: {avg_of_output_token_count} ")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



 avg_of_output_token_count: 257.50905902980713 


In [7]:
"""## Inference"""



# change me!

path = "/home/trkosire/LLM_RESEARCH/Finance/out_OpenELM-270M_LR-5e-05_OA_bf130067-0855-427c-ba89-f63b91981af7/checkpoint-3250"
model_path = path 

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    # device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    use_fast=False)

model.to("cuda:0")
def prompt(question, debug=False, max_new_tokens=250):
    messages = [
        {"role": "user", "content": question},
    ]

    input_tokens = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda:0")
    output_tokens = model.generate(input_tokens, max_new_tokens=max_new_tokens,do_sample=True, top_k=30,
                                 top_p=0.8, no_repeat_ngram_size = 5, num_return_sequences=1)

    if debug:
        for tok in output_tokens[0]:
            print(tok, tokenizer.decode(tok))

    output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    rem_extra = output.find("assistant") + len("assistant")
    modified_output = output[rem_extra:].strip()

    return output, modified_output



/home/trkosire/LLM_RESEARCH/Finance/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
prompt("Use your finanacial knowledge and give the answer the following question, What is investing?")


('user\nUse your finanacial knowledge and give the answer the following question, What is investing?\nassistant\nThe only time you should invest in the market is when you need to get some returns. This is when you want to take on more risk, and when you want to get the best deal you can. If you are interested in investing in the stock market, I would start with a Roth IRA and put your money into it. That is the easiest way to get the most out of it.',
 'The only time you should invest in the market is when you need to get some returns. This is when you want to take on more risk, and when you want to get the best deal you can. If you are interested in investing in the stock market, I would start with a Roth IRA and put your money into it. That is the easiest way to get the most out of it.')

In [19]:
eval_df_with_sys_prompt = test_df.copy()

In [20]:
eval_df_with_sys_prompt['input_with_sys_prompt'] = eval_df_with_sys_prompt['instruction'] + "\n" + eval_df_with_sys_prompt['input']

In [23]:
eval_df_with_sys_prompt.columns

Index(['input', 'output', 'instruction', 'messages', 'input_with_sys_prompt'], dtype='object')

In [24]:
eval_df_with_sys_prompt[['original_ouput_with_sys_prompt', 'modified_output_with_sys_prompt']] = eval_df_with_sys_prompt['input_with_sys_prompt'].apply(prompt).apply(pd.Series)

In [25]:
eval_df_with_sys_prompt.columns

Index(['input', 'output', 'instruction', 'messages', 'input_with_sys_prompt',
       'original_ouput_with_sys_prompt', 'modified_output_with_sys_prompt'],
      dtype='object')

In [27]:
for index in range(len(eval_df_with_sys_prompt)):
    print(f"\n Question: ", eval_df_with_sys_prompt['input_with_sys_prompt'][index])
    print(f"\n Answer: ", eval_df_with_sys_prompt['modified_output_with_sys_prompt'][index])
    print()
    print()


 Question:  Utilize your financial knowledge, give your answer or opinion to the input question or subject . Answer format is not limited.
Has anyone compared an in-person Tax Advisor to software like Turbo Tax?

 Answer:  I'm sure there are plenty of good ones, but it would be difficult to give an honest opinion without actually talking to the person.  I did find a few on Yelp that said they were a great experience, but they were mostly negative.  There is nothing wrong with using the software, but you have to consider the time spent, and the fact that you are not in the same state at the same time.  This is a good way to waste money.  You can find plenty of good ones at taxes.com, but I did not think they were that great.  I think it depends on your situation, but I'd recommend you talk to someone.



 Question:  Offer your insights or judgment on the input financial query or topic using your financial expertise. Reply as normal question answering
Understanding the phrase “afford to

In [28]:
eval_df_with_sys_prompt.to_csv("eval_file_with_system_prompt.csv", index = False)
eval_df_with_sys_prompt.to_pickle("eval_file_with_system_prompt.pkl")